# Building operators

dynamite is designed to study systems of interacting spin-1/2 particles, with arbitrary interactions. It can be used for both time evolution under a Hamiltonian, as well as solving for eigenvalues and eigenstates. In this notebook we will learn the basics of building quantum operators. 

The full documentation for operators in dynamite can be found [here](https://dynamite.readthedocs.io/en/stable/dynamite.operators.html).

### The Pauli matrices

The building blocks for operators in dynamite are the Pauli matrices:

In [ ]:
from dynamite.operators import sigmax, sigmay, sigmaz, identity

Try creating a $\sigma^x$ operator:

In [ ]:
# run this cell to evaluate the sigmax() function
sigmax()

In a Jupyter notebook, dynamite displays the operator using TeX. If you want to see the matrix itself you can make a NumPy array out of it:

In [ ]:
sigmax().to_numpy(sparse=False)

Nice. You may have noticed the subscript 0 on the TeX output. This denotes the site the operator is on: when called with no arguments, the Pauli operators are created on site 0. It is implied that the operator is tensored with the identity operator on all other sites.

You can specify a different site for the operator by passing an integer to the function:

In [ ]:
# make a Pauli X on site 2
sigmax(2)

### Combining operators

Operators combine in the way that you would expect, for example via addition with `+` and multiplication with `*`.

Try making a two-body XX operator on sites 2 and 3, by multiplying together two Pauli X's:

In [ ]:
sigmax(2)*sigmax(3)

You can also add operators to each other, and add and multiply constants (which are interpreted as the identity times that constant). Play around with it in the above cell if you like!

You can manipulate operators more or less like you would any other arithmetic object in Python. For example, the Python builtin `sum`:

In [ ]:
sum(sigmax(i) for i in range(5))

We should note, though, that there's actually a better way to translate operators across a 1D spin chain: the `index_sum` and `index_product` functions. Before we get to those, let's look at how to specify the length of the spin chain.

### Specifying the spin chain length

It's usually most convenient to set the spin chain length globally at the beginning of your program:

In [ ]:
from dynamite import config
config.L = 8

In [ ]:
z = sigmaz()  # this operator inherits the spin chain length from config.L

z.L           # output the length of spin chain this operator lives on

If needed, you can also get and set the length of the spin chain of individual operators via the `L` property:

In [ ]:
z.L = 8        # set the length

OK, now we're ready to learn about translating operators across the chain.

### Translating operators across a 1D chain

Using 1D spin chains in dynamite is so common that there are some convenience functions for it (which are also faster than `sum`, for large operators).

In [ ]:
from dynamite.operators import index_sum, index_product

Try calling the `index_sum` function on a `sigmax()` operator:

In [ ]:
# translate a Pauli X operator across the spin chain
index_sum(sigmax())

Because we set `config.L` to 8, we now have operators on sites 0 through 7 (all of them).

Note that if we pass a many-body operator, the sum will be adjusted accordingly. Try passing a two-body XX operator on sites 0 and 1 to `index_sum`:

In [ ]:
# translate an XX operator across the spin chain
index_sum(sigmax(0)*sigmax(1))

Note that for our 8 site chain, the sum only goes from 0 to 6 this time, so we don't go off the end of the chain.
You can try adding the parameter `boundary='closed'` to `index_sum` to get closed boundary conditions!

For complicated operators, you may want to inspect the individual terms by printing the output of the `.table()` function: 

In [ ]:
op = index_sum(sigmax(0)*sigmax(1))
print(op.table())

With closed boundary conditions, we see the "wrap-around" term:

In [ ]:
op = index_sum(sigmax(0)*sigmax(1), boundary='closed')
print(op.table())

As you might expect, `index_product` works like `index_sum`, but it's a product instead of a sum:

In [ ]:
index_product(sigmaz())

### Exercise: Putting it all together

Try writing a function that implements a classic Hamiltonian: the transverse field Ising model! The Hamiltonian we want is

$$H = J \sum_i \sigma^z_i \sigma^z_{i+1} + h \sum_i \sigma^x_i$$

In [ ]:
def tfim(J, h):
    H = identity()  # TODO: your code here to define and return H
    return H

# test it out! Should return the correct Hamiltonian, which will output in TeX
# note that config.L is still set to 8 from above
tfim(1, 0.5)

In [ ]:
print(tfim(1, 0.5).table())

## More about operators

### Working with sparse matrices

A key feature of dynamite is that it deals with matrices in sparse format---that is, it only stores non-zero matrix elements (or can be configured to not store the matrix elements at all, see the notebook about "shell" matrices!)

For example, let's look at the matrix underlying your transverse field Ising model operator.

In [ ]:
config.L = 5  # so it's not crazy huge
H = tfim(1, 0.5)
H.to_numpy(sparse=False)

It's clearly a lot of zeros, but it's kind of hard to visualize what's going on here---we can use the `.spy()` method to see the nonzero structure of the matrix:

In [ ]:
H.spy()

Black squares represent nonzero terms; the white ones are zero. This is why dynamite uses a sparse representation, to avoid storing all those extra zeros.

If you want a NumPy representation of an operator in sparse format, you can simply call `.to_numpy()` without the `sparse=False` option.

In [ ]:
H.to_numpy()

For most computations you will want to use dynamite's built-in algorithms rather than converting the matrix to NumPy, but if there is a situation where you want to use NumPy, you can always use dynamite just to build the matrix!

## Up next

[Continue to notebook 2](2-States.ipynb) to learn about the `State` class.